In [25]:
import numpy as np
import pandas as pd
from math import ceil
from copy import deepcopy
from abc import ABC, abstractmethod

In [4]:
# how to create animal:

# pig = Animal('Pig')

# what I want to happen:

# animal.target_func = get_target_func(*species_data.loc[i, 'Target':'Target_pos'])
# animal.effect_func = get_effect_func(species, lvl, *data.loc['Effect':'Effect_repeat'])

# animal.target_func(own_team, opp_team/shop) --> [targets]
# animal.effect_func([targets], own_team, opp_team/shop, n_triggers) --> does the effect

In [1]:
# for Summon
def Tier1Pet(owner, gamestate, *args):  # sabertooth tiger
    return Pet(np.random.choice(allpacks_animal_dict[1]), atk=8*owner.lvl, hp=8*owner.lvl, lvl=owner.lvl)

def Tier3Pet(owner, gamestate, *args):  #spider
    return Pet(np.random.choice(gamestate.animal_dict[3]), atk=2*owner.lvl, hp=2*owner.lvl, lvl=owner.lvl)

def Tier6Pet(owner, gamestate, *args):  # eagle
    return Pet(np.random.choice(allpacks_animal_dict[6]), lvl=owner.lvl)

def PreviousTier(owner, gamestate, in_team1, lvl):  # stork
    return Pet(np.random.choice(allpacks_animal_dict[max(1, gamestate.current_tier - 1)]), lvl=owner.lvl)

def Chick(owner, *args):  # rooster
    return Pet('Chick', atk=max(1, round(owner.atk / 2)), hp=1, lvl=owner.lvl)

def FaintAbilityPet(*args):  # orca
    return Pet(np.random.choice(faint_species), lvl=1)

def RandomFriendPet(owner, gamestate, in_team1, lvl):  # tapir
    own_team = gamestate.team1 if in_team1 else gamestate.team2
    valid_species = list(filter(lambda x: x not in ['Tapir', 'Orca'], [pet.species for pet in own_team]))
    if len(valid_species) == 0:
        return 
    else:
        return Pet(np.random.choice(valid_species), lvl=lvl)
    
def HoneySummon(owner, *args):
    return Pet('Bee', lvl=owner.lvl)
    
def MushroomSummon(owner, *args):
    return Pet(owner.species, lvl=owner.lvl, atk=1, hp=1)

def PopcornsSummon(owner, *args):
    return Pet(np.random.choice(allpacks_animal_dict[owner.tier]), lvl=owner.lvl)

# for ApplyStatus
def OwnFood(pet):
    return pet.status

# for DealDamage (lynx)
def FriendLvlSum(owner, own_team):
    return sum([pet.lvl for pet in own_team.exclude(owner.pos)])

# for ModifyStats (moose)
def LowestShopTier(owner, shop):
    assert isinstance(shop, Shop)
    return min([pet.tier for pet in shop.team2])

# for ModifyStats (ostrich)
def NTier5Or6PetsInShop(owner, shop):
    assert isinstance(shop, Shop)
    return len(list(filter(lambda pet: pet.tier >= 5, shop.team2)))

# for ModifyStats (stegosaurus)
def TurnNumber(owner, shop):
    assert isinstance(shop, Shop)
    return shop.current_turn


In [23]:
class EffectFunction(ABC):
    def __init__(self, owner, from_, info, repeat):
        self.owner = owner
        self.repeat = repeat
    
    def __call__(self, targets, gamestate, in_team1):
        for _ in range(self.repeat):
            output = self.function(self, targets, gamestate, in_team1)
        
class AddShopFood(EffectFunction):
    def __init__(self, owner, from_, info, repeat):
        super().__init__(owner, from_, info, repeat)
        def function(self, targets, gamestate, in_team1):
            gamestate.foods.add_food_by_tier(gamestate.current_tier)
            for _ in range(max(0, len(gamestate.team2) + len(gamestate.foods) - Shop.max_size)):
                gamestate.team2.drop(-1)
        self.function = function
        
class ApplyStatus(EffectFunction):
    def __init__(self, owner, from_, info, repeat):
        super().__init__(owner, from_, info, repeat)
        if info.get('status'):
            def function(self, targets, gamestate, in_team1):
                for pet in targets:
                    pet.status = Status(info['status'])
        elif callable(info.get('status_func')):
            def function(self, targets, gamestate, in_team1):
                for pet in targets:
                    pet.status = info['status_func'](self.owner)
        else:
            assert False, 'ApplyStatus got a bad Effect_info'
        self.function = function
        
class BuffShop(EffectFunction):
    def __init__(self, owner, from_, info, repeat):
        super().__init__(owner, from_, info, repeat)
        def function(self, targets, gamestate, in_team1):
            gamestate.team2.perm_atk_buff += info['add_atk']
            gamestate.team2.perm_hp_buff += info['add_hp']
        self.function = function

class DealDamage(EffectFunction):
    def __init__(self, owner, from_, info, repeat):
        super().__init__(owner, from_, info, repeat)
        def function(self, targets, gamestate, in_team1):
            if info.get('damage'):
                damage = info['damage']
                if self.owner.status.name == 'Pineapple':
                    damage += 2
            elif info.get('damage_perc'):
                damage = round(info['damage_perc']/100 * self.owner.atk)
            elif info.get('damage_func'):
                damage = info['damage_func'](self.owner, gamestate.team1 if in_team1 else gamestate.team2)
                
            for pet in targets:
                pet.take_damage(max(1, damage))
        self.function = function
        
class DiscountFood(EffectFunction):
    def __init__(self, owner, from_, info, repeat):
        super().__init__(owner, from_, info, repeat)
        def function(self, targets, gamestate, in_team1):
            for food in gamestate.foods:
                food.cost = max(0, food.cost - info['discount'])
        self.function = function

class DiscountPets(EffectFunction):
    def __init__(self, owner, from_, info, repeat):
        super().__init__(owner, from_, info, repeat)
        def function(self, targets, gamestate, in_team1):
            for pet in targets:
                assert isinstance(pet, ShopAnimal), f'DiscountPets did not get ShopAnimals as targets'
                pet.cost = max(0, pet.cost - info['discount'])
        self.function = function
        
class Faint(EffectFunction):
    def __init__(self, owner, from_, info, repeat):
        super().__init__(owner, from_, info, repeat)
        def function(self, targets, gamestate, in_team1):
            for pet in targets:
                pet.hp = 0
                pet.trigger_dict['Faint'] = 1
        self.function = function

class GainExperience(EffectFunction):
    def __init__(self, owner, from_, info, repeat):
        super().__init__(owner, from_, info, repeat)
        def function(self, targets, gamestate, in_team1):
            for pet in targets:
                # for some reason, gaining exp always coincides with a 1,1 buff, maybe needs update
                if pet.lvl in [1,2]:
                    pet.atk += info['exp']
                    pet.hp += info['exp']
                pet.exp += info['exp']
        self.function = function

class GainGold(EffectFunction):
    def __init__(self, owner, from_, info, repeat):
        super().__init__(owner, from_, info, repeat)
        def function(self, targets, gamestate, in_team1):
            gamestate.gold += info['gold']
        self.function = function

class ModifyStats(EffectFunction):
    def __init__(self, owner, from_, info, repeat):
        super().__init__(owner, from_, info, repeat)
        def function(self, targets, gamestate, in_team1):
            mult = info.get('mult_func', lambda *args: True)(self.owner, gamestate)
            for pet in targets:
                pet.buff_stats(info.get('add_atk', 0) * mult, info.get('add_hp', 0) * mult, temp=info.get('temp'))  
        self.function = function

class NoEffect(EffectFunction):
    def __init__(self, owner, from_, info, repeat):
        super().__init__(owner, from_, info, repeat)
        self.function = lambda: None

class Push(EffectFunction):
    def __init__(self, owner, from_, info, repeat):
        super().__init__(owner, from_, info, repeat)
        def function(self, targets, gamestate, in_team1):
            opp_team = gamestate.team2 if in_team1 else gamestate.team1
            for pet in targets:
                opp_team.drop(pet.pos)
                opp_team.insert(pet, pet.pos + info['spaces_pushed'])
                pet.trigger_dict['Pushed'] += 1
        self.function = function

class ReduceHealth(EffectFunction):
    def __init__(self, owner, from_, info, repeat):
        super().__init__(owner, from_, info, repeat)
        def function(self, targets, gamestate, in_team1):
            for pet in targets:
                pet.hp = ceil(pet.hp * info['set_hp_perc'] / 100)
        self.function = function

class ReplaceShopFood(EffectFunction):
    def __init__(self, owner, from_, info, repeat):
        super().__init__(owner, from_, info, repeat)
        def function(self, targets, gamestate, in_team1):
            new_foods = [Food(name) for name in info['replace_with']]
            # if too many foods in shop, overwrite pets
            for _ in range(max(0, len(gamestate.team2) + len(new_foods) - Shop.max_size)):
                gamestate.team2.drop(-1)
            gamestate.foods.pets = new_foods
        self.function = function
        
class RollShop(EffectFunction):
    def __init__(self, owner, from_, info, repeat):
        super().__init__(owner, from_, info, repeat)
        def function(self, targets, gamestate, in_team1):
            gamestate.roll(free_roll=True)
        self.function = function

class SetLevel(EffectFunction):
    def __init__(self, owner, from_, info, repeat):
        super().__init__(owner, from_, info, repeat)
        def function(self, targets, gamestate, in_team1):
            for pet in targets:
                pet.lvl = info['lvl']
        self.function = function

class ShuffleTeams(EffectFunction):
    def __init__(self, owner, from_, info, repeat):
        super().__init__(owner, from_, info, repeat)
        def function(self, targets, gamestate, in_team1):
            gamestate.team1.shuffle()
            gamestate.team2.shuffle()
        self.function = function

class StealFood(EffectFunction):
    def __init__(self, owner, from_, info, repeat):
        super().__init__(owner, from_, info, repeat)
        def function(self, targets, gamestate, in_team1):
            assert len(targets) <= 1, f'StealFood got 2 targets, needs disambiguation'
            target = targets[0]
            if target.status.name != 'NoStatus' and target.status.n_charges > 0:
                self.owner.status = targets[0].status
                targets[0].status = Status('NoStatus')
        self.function = function

class StealShopFood(EffectFunction):
    def __init__(self, owner, from_, info, repeat):
        super().__init__(owner, from_, info, repeat)
        def function(self, targets, gamestate, in_team1):
            ind = np.random.choice(range(len(gamestate.foods)))
            gamestate.foods.drop(ind).use(self.owner, gamestate)
        self.function = function

# might need to add custom atk/hp values from Effect_info
class Summon(EffectFunction):
    def __init__(self, owner, from_, info, repeat):
        super().__init__(owner, from_, info, repeat)
        # create helper function make_pet
        if info.get('pet'):
            make_pet = lambda *args: Pet(info['pet'], lvl = info.get('lvl'))
        elif info.get('pet_func'):
            make_pet = info['pet_func']
        else:
            assert False, 'Summon got a bad Effect_info (no pet or pet_func)'
            
        in_opp_team = info.get('in_opp_team', False)
        def function(self, targets, gamestate, in_team1):
            assert len(targets) <= 1, f'Summon got >=2 targets, needs disambiguation'
            if isinstance(gamestate, Shop):
                return  # e.g. when rat is pilled
            # still summon if enemy target dies
            summon_pos = targets[0].pos if len(targets) > 0 else 0
            # do an xor here
            summon_team = gamestate.team2 if in_opp_team == in_team1 else gamestate.team1
            to_summon = make_pet(self.owner, gamestate, in_team1, info.get('lvl'))
            
            if to_summon is None:
                return
            if len(summon_team) < Team.max_size:
                summon_team.insert(to_summon, summon_pos)
                
        self.function = function

class Swallow(EffectFunction):
    def __init__(self, owner, from_, info, repeat):
        super().__init__(owner, from_, info, repeat)
        def function(self, targets, gamestate, in_team1):
            assert len(targets) == 1, f'Swallow got 2 targets, needs disambiguation'
            swallowed = targets[0]
            self.owner.temp_abilities = {lvl: [Ability(self.owner, 'Faint', 'Self', {}, [Self], -1, (), 
                                                       [Summon], [NoTarget], 
                                                       {'pet': swallowed.species, 'lvl': lvl}, 
                                                       1, n_charges = 1)] for lvl in [1, 2, 3]}
            swallowed.hp = 0
            swallowed.trigger_dict['Faint'] = 1
        self.function = function
        
class SwapAttackHealth(EffectFunction):
    def __init__(self, owner, from_, info, repeat):
        super().__init__(owner, from_, info, repeat)
        def function(self, targets, gamestate, in_team1):
            for pet in targets:
                temp = pet.atk
                pet.atk = pet.hp
                pet.hp = temp
        self.function = function

class SwapStats(EffectFunction):
    def __init__(self, owner, from_, info, repeat):
        super().__init__(owner, from_, info, repeat)
        def function(self, targets, gamestate, in_team1):
            if len(targets) < 2:
                return
            if len(targets) == 2:
                temp_atk, temp_hp = targets[0].atk, targets[0].hp
                targets[0].atk, targets[0].hp = targets[1].atk, targets[1].hp
                targets[1].atk, targets[1].hp = temp_atk, temp_hp
            else:
                assert False, f'SwapStats got more than 2 targets: {targets}'
        self.function = function

# a future update might need to switch target and effect_from in pets_df
class TransferAbility(EffectFunction):
    def __init__(self, owner, from_, info, repeat):
        super().__init__(owner, from_, info, repeat)
        def function(self, targets, gamestate, in_team1):
            assert len(targets) <= 1, f'TransferAbility got 2 targets, needs disambiguation'
            target = targets[0]
            self.owner.temp_abilities = {lvl: make_abl_list(self.owner, *pets_df.loc[(target.species, lvl), 'Trigger':'N_charges'])
                                       for lvl in [1, 2, 3]}
        self.function = function

class TransferStats(EffectFunction):
    def __init__(self, owner, from_, info, repeat):
        super().__init__(owner, from_, info, repeat)
        def function(self, targets, gamestate, in_team1):
            assert callable(from_), 'TransferStats got a bad Effect_from (not callable)'
            target_func = from_(self.owner, np.nan, np.nan)
            pets_from = target_func(gamestate, in_team1)
            assert len(pets_from) <= 1, f'TransferStats got more than 1 sources, needs disambiguation'
            if len(pets_from) == 0:
                return 0
            pet_from = pets_from[0]
            
            set_atk = round(info.get('set_atk_perc', 0)/100 * pet_from.atk)
            set_hp = round(info.get('set_hp_perc', 0)/100 * pet_from.hp)
            add_atk = round(info.get('add_atk_perc', 0)/100 * pet_from.atk)
            add_hp = round(info.get('add_hp_perc', 0)/100 * pet_from.hp)
            
            if set_atk: 
                for pet in targets:
                    pet.atk = set_atk
            elif add_atk:
                for pet in targets:
                    pet.buff_stats(atk=add_atk, temp=info.get('temp'))
            
            if set_hp:
                for pet in targets:
                    pet.hp = set_hp
            elif add_hp:
                for pet in targets:
                    pet.buff_stats(hp=add_hp, temp=info.get('temp'))        

        self.function = function

# a future update might need pet_func compatibility
class Transform(EffectFunction):
    def __init__(self, owner, from_, info, repeat):
        super().__init__(owner, from_, info, repeat)
        def function(self, targets, gamestate, in_team1):
            own_team = gamestate.team1 if in_team1 else gamestate.team2
            new_pet = Pet(info['pet'], lvl=info['lvl'])
            new_pet.trigger_dict['Transformed'] = 1
            own_team[self.owner.pos] = new_pet
            new_pet.pos = self.owner.pos
        self.function = function



In [24]:
# t1 = Team([Animal(name) for name in 'Ant Badger Cat Dodo Eagle'.split()])
# t2 = Team([Animal(name) for name in ['Pig']*5])
# d = t1[3]
# d.effect_func = TransferStats(d, *pets_df.loc[('Butterfly', 1), 'Effect_from':'Effect_repeat'])
# d.effect_func([t1[2]], t1, t2)